# 14장 배낭 문제와 그래프 최적화 문제

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/python4daml/blob/main/14장.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

## 14.1 배낭 문제

### 14.1.1 탐욕 알고리즘

예제 14-1 Item 클래스

In [1]:
class Item(object): 
    def __init__(self, n, v, w): 
        self._name = n 
        self._value = v 
        self._weight = w 
    def get_name(self): 
        return self._name 
    def get_value(self): 
        return self._value 
    def get_weight(self): 
        return self._weight 
    def __str__(self): 
        return f'<{self._name}, {self._value}, {self._weight}>' 

def value(item): 
    return item.get_value() 

def weight_inverse(item): 
    return 1.0/item.get_weight() 

def density(item): 
    return item.get_value()/item.get_weight() 

예제 14-2 탐욕 알고리즘 구현

In [2]:
def greedy(items, max_weight, key_function): 
    """items는 리스트, max_weight >= 0, 
       key_function는 items의 원소를 숫자에 매핑한다고 가정합니다""" 
    items_copy = sorted(items, key=key_function, reverse = True) 
    result = [] 
    total_value, total_weight = 0.0, 0.0 
    for i in range(len(items_copy)): 
        if (total_weight + items_copy[i].get_weight()) <= max_weight: 
            result.append(items_copy[i]) 
            total_weight += items_copy[i].get_weight() 
            total_value += items_copy[i].get_value() 
    return (result, total_value) 

예제 14-3 탐욕 알고리즘으로 물건 선택하기

In [3]:
def build_items(): 
    names = ['시계','그림','라디오','꽃병','책','컴퓨터'] 
    values = [175,90,20,50,10,200] 
    weights = [10,9,4,2,1,20] 
    Items = [] 
    for i in range(len(values)): 
        Items.append(Item(names[i], values[i], weights[i])) 
    return Items 

def test_greedy(items, max_weight, key_function): 
    taken, val = greedy(items, max_weight, key_function) 
    print('선택한 물건의 총 가치:', val) 
    for item in taken: 
        print(' ', item) 

def test_greedys(max_weight = 20): 
    items = build_items() 
    print('가치 기준 탐욕 알고리즘을 사용해', max_weight, '크기의 배낭을 채우기') 
    test_greedy(items, max_weight, value) 
    print('\n무게 기준 탐욕 알고리즘을 사용해', max_weight, '크기의 배낭을 채우기') 
    test_greedy(items, max_weight, weight_inverse) 
    print('\n밀도 기준 탐욕 알고리즘을 사용해', max_weight, '크기의 배낭을 채우기') 
    test_greedy(items, max_weight, density) 

In [4]:
test_greedys()

가치 기준 탐욕 알고리즘을 사용해 20 크기의 배낭을 채우기
선택한 물건의 총 가치: 200.0
  <컴퓨터, 200, 20>

무게 기준 탐욕 알고리즘을 사용해 20 크기의 배낭을 채우기
선택한 물건의 총 가치: 170.0
  <책, 10, 1>
  <꽃병, 50, 2>
  <라디오, 20, 4>
  <그림, 90, 9>

밀도 기준 탐욕 알고리즘을 사용해 20 크기의 배낭을 채우기
선택한 물건의 총 가치: 255.0
  <꽃병, 50, 2>
  <시계, 175, 10>
  <책, 10, 1>
  <라디오, 20, 4>


### 14.1.2 0/1 배낭 문제의 최적 솔루션

In [5]:
def get_binary_rep(n, num_digits): 
    """n과 num_digits은 음수가 아닌 정수로 가정합니다. 
       n의 이진 표현을 num_digits 길이의 문자열로 반환합니다""" 
    result = '' 
    while n > 0: 
        result = str(n%2) + result 
        n = n//2 
    if len(result) > num_digits: 
        raise ValueError('num_digits가 부족합니다') 
    for i in range(num_digits - len(result)): 
        result = '0' + result 
    return result 

def gen_powerset(L): 
    """L은 리스트로 가정합니다. 
       L에 있는 원소로 가능한 모든 조합을 담은 리스트의 리스트를 반환합니다.
       예를 들어 L이 [1, 2]이면 [], [1], [2], [1, 2]를 원소로 가진 리스트를 반환합니다""" 
    powerset = [] 
    for i in range(0, 2**len(L)): 
        bin_str = get_binary_rep(i, len(L)) 
        subset = [] 
        for j in range(len(L)): 
            if bin_str[j] == '1': 
                subset.append(L[j]) 
        powerset.append(subset) 
    return powerset 

예제 14-4 0/1 배낭 문제의 단순하지만 최적인 솔루션

In [6]:
def choose_best(pset, max_weight, get_val, get_weight): 
    best_val = 0.0 
    best_set = None 
    for items in pset: 
        items_val = 0.0 
        items_weight = 0.0 
        for item in items: 
            items_val += get_val(item) 
            items_weight += get_weight(item) 
        if items_weight <= max_weight and items_val > best_val: 
            best_val = items_val 
            best_set = items 
    return (best_set, best_val) 

def test_best(max_weight = 20): 
    items = build_items() 
    pset = gen_powerset(items) 
    taken, val = choose_best(pset, max_weight, Item.get_value, 
                             Item.get_weight) 
    print('선택한 물건의 총 가치:', val) 
    for item in taken: 
        print(item) 

In [7]:
test_best()

선택한 물건의 총 가치: 275.0
<시계, 175, 10>
<그림, 90, 9>
<책, 10, 1>


## 14.2 그래프 최적화 문제

예제 14-5 노드와 에지

In [8]:
class Node(object): 
    def __init__(self, name): 
        """name은 문자열이라고 가정합니다""" 
        self._name = name 
    def get_name(self): 
        return self._name 
    def __str__(self): 
        return self._name 

class Edge(object): 
    def __init__(self, src, dest): 
        """src와 dest는 노드라고 가정합니다""" 
        self._src = src 
        self._dest = dest 
    def get_source(self): 
        return self._src 
    def get_destination(self): 
        return self._dest 
    def __str__(self): 
        return self._src.get_name() + '->' + self._dest.get_name() 

class Weighted_edge(Edge): 
    def __init__(self, src, dest, weight = 1.0): 
        """src와 dest는 노드이고, weight는 숫자라고 가정합니다""" 
        self._src = src 
        self._dest = dest 
        self._weight = weight 
    def get_weight(self): 
        return self._weight 
    def __str__(self): 
        return (f'{self._src.get_name()}->({self._weight})' + 
                f'{self._dest.get_name()}') 

예제 14-6 `Graph`와 `Digraph` 클래스

In [9]:
class Digraph(object): 
    #_nodes는 그래프에 있는 노드의 리스트입니다.
    #_edges는 각 노드를 자식 노드 리스트에 매핑한 딕셔너리입니다.
    def __init__(self): 
        self._nodes = [] 
        self._edges = {} 
    def add_node(self, node): 
        if node in self._nodes: 
            raise ValueError('Duplicate node') 
        else: 
            self._nodes.append(node) 
            self._edges[node] = [] 
    def add_edge(self, edge): 
        src = edge.get_source() 
        dest = edge.get_destination() 
        if not (src in self._nodes and dest in self._nodes): 
            raise ValueError('Node not in graph') 
        self._edges[src].append(dest) 
    def children_of(self, node): 
        return self._edges[node] 
    def has_node(self, node): 
        return node in self._nodes 
    def __str__(self): 
        result = '' 
        for src in self._nodes: 
            for dest in self._edges[src]: 
                result = (result + src.get_name() + '->' 
                          + dest.get_name() + '\n') 
        return result[:-1] #마지막 줄바꿈을 제외합니다

class Graph(Digraph): 
    def add_edge(self, edge): 
        Digraph.add_edge(self, edge) 
        rev = Edge(edge.get_destination(), edge.get_source()) 
        Digraph.add_edge(self, rev) 

### 14.2.1 고전 그래프 문제

### 14.2.2 최단 경로: 깊이 우선 탐색과 너비 우선 탐색

예제 14-7 깊이 우선 탐색 최단 경로 알고리즘

In [10]:
def print_path(path): 
    """path는 노드의 리스트라고 가정합니다""" 
    result = '' 
    for i in range(len(path)): 
        result = result + str(path[i]) 
        if i != len(path) - 1: 
            result = result + '->' 
    return result 

def DFS(graph, start, end, path, shortest, to_print = False): 
    """Assumes graph는 Digraph, start와 end는 노드, 
       path와 shortest는 노드의 리스트로 가정합니다.
       graph에서 start부터 end까지 최단 경로를 반환합니다""" 
    path = path + [start] 
    if to_print: 
        print('현재 DFS 경로:', print_path(path)) 
    if start == end: 
        return path 
    for node in graph.children_of(start): 
        if node not in path: #순환을 피합니다
            if shortest == None or len(path) < len(shortest): 
                new_path = DFS(graph, node, end, path, shortest, to_print) 
                if new_path != None: 
                    shortest = new_path 
    return shortest 

def shortest_path(graph, start, end, to_print = False): 
    """graph는 Digraph, start와 end는 노드라고 가정합니다.
       그래프에서 start부터 end까지 최단 경로를 반환합니다""" 
    return DFS(graph, start, end, [], None, to_print) 

예제 14-8 깊이 우선 탐색 코드 테스트

In [11]:
def test_SP(): 
    nodes = [] 
    for name in range(6): #6개 노드를 만듭니다
        nodes.append(Node(str(name))) 
    g = Digraph() 
    for n in nodes: 
        g.add_node(n) 
    g.add_edge(Edge(nodes[0],nodes[1])) 
    g.add_edge(Edge(nodes[1],nodes[2])) 
    g.add_edge(Edge(nodes[2],nodes[3])) 
    g.add_edge(Edge(nodes[2],nodes[4])) 
    g.add_edge(Edge(nodes[3],nodes[4])) 
    g.add_edge(Edge(nodes[3],nodes[5])) 
    g.add_edge(Edge(nodes[0],nodes[2])) 
    g.add_edge(Edge(nodes[1],nodes[0])) 
    g.add_edge(Edge(nodes[3],nodes[1])) 
    g.add_edge(Edge(nodes[4],nodes[0])) 
    sp = shortest_path(g, nodes[0], nodes[5], to_print = True) 
    print('DFS가 찾은 최단 경로:', print_path(sp))

In [12]:
test_SP()

현재 DFS 경로: 0
현재 DFS 경로: 0->1
현재 DFS 경로: 0->1->2
현재 DFS 경로: 0->1->2->3
현재 DFS 경로: 0->1->2->3->4
현재 DFS 경로: 0->1->2->3->5
현재 DFS 경로: 0->1->2->4
현재 DFS 경로: 0->2
현재 DFS 경로: 0->2->3
현재 DFS 경로: 0->2->3->4
현재 DFS 경로: 0->2->3->5
현재 DFS 경로: 0->2->3->1
현재 DFS 경로: 0->2->4
DFS가 찾은 최단 경로: 0->2->3->5


**뇌풀기 문제**

In [13]:
class Wgraph(Digraph): 
    def __init__(self): 
        super().__init__()
        self._weighted_edges = [] 
    def add_edges(self, edges):
        self._weighted_edges = edges
        for edge in edges:
            self.add_edge(edge)
    def get_weights(self, path):
        weight_sum = 0
        parent_node = path[0]
        for child_node in path[1:]:
            for edge in self._weighted_edges:
                if edge._src == parent_node and edge._dest == child_node:
                    weight_sum += edge.get_weight()
                    break
            parent_node = child_node
        return weight_sum

def WDFS(graph, start, end, path, shortest, to_print = False): 
    """Assumes graph는 Digraph, start와 end는 노드, 
       path와 shortest는 노드의 리스트로 가정합니다.
       graph에서 start부터 end까지 최단 경로를 반환합니다""" 
    path = path + [start] 
    if to_print: 
        print('현재 DFS 경로:', print_path(path), '\t', 
              '가중치 합:', graph.get_weights(path)) 
    if start == end:
        return path 
    for node in graph.children_of(start):
        if node in path: #순환을 피합니다
            continue
        new_path = WDFS(graph, node, end, path, shortest, to_print) 
        if shortest == None or (new_path != None and \
           graph.get_weights(new_path) < graph.get_weights(shortest)): 
            shortest = new_path 
    return shortest 

def test_WSP(): 
    nodes = [] 
    for name in range(6): #6개 노드를 만듭니다
        nodes.append(Node(str(name))) 
    g = Wgraph() 
    for n in nodes: 
        g.add_node(n)
    edges = [Weighted_edge(nodes[0],nodes[1],1),
             Weighted_edge(nodes[1],nodes[2],2),
             Weighted_edge(nodes[2],nodes[3],1),
             Weighted_edge(nodes[2],nodes[4],1),
             Weighted_edge(nodes[3],nodes[4],1),
             Weighted_edge(nodes[3],nodes[5],2),
             Weighted_edge(nodes[0],nodes[2],4),
             Weighted_edge(nodes[1],nodes[0],1),
             Weighted_edge(nodes[3],nodes[1],1),
             Weighted_edge(nodes[4],nodes[0],1)
             ]
    g.add_edges(edges) 
    sp = WDFS(g, nodes[0], nodes[5], [], None, to_print = True) 
    print('DFS가 찾은 최단 경로:', print_path(sp))

test_WSP()

현재 DFS 경로: 0 	 가중치 합: 0
현재 DFS 경로: 0->1 	 가중치 합: 1
현재 DFS 경로: 0->1->2 	 가중치 합: 3
현재 DFS 경로: 0->1->2->3 	 가중치 합: 4
현재 DFS 경로: 0->1->2->3->4 	 가중치 합: 5
현재 DFS 경로: 0->1->2->3->5 	 가중치 합: 6
현재 DFS 경로: 0->1->2->4 	 가중치 합: 4
현재 DFS 경로: 0->2 	 가중치 합: 4
현재 DFS 경로: 0->2->3 	 가중치 합: 5
현재 DFS 경로: 0->2->3->4 	 가중치 합: 6
현재 DFS 경로: 0->2->3->5 	 가중치 합: 7
현재 DFS 경로: 0->2->3->1 	 가중치 합: 6
현재 DFS 경로: 0->2->4 	 가중치 합: 5
DFS가 찾은 최단 경로: 0->1->2->3->5


예제 14-9 너비 우선 탐색 최단 경로 알고리즘

In [14]:
def BFS(graph, start, end, to_print = False): 
    """graph는 Digraph, start와 end는 노드라고 가정합니다.
       graph에서 start부터 end까지 최단 경로를 반환합니다""" 
    init_path = [start] 
    path_queue = [init_path] 
    while len(path_queue) != 0: 
        #path_queue의 첫 번째 원소를 추출합니다
        tmp_path = path_queue.pop(0) 
        if to_print: 
            print('현재 BFS 경로:', print_path(tmp_path)) 
        last_node = tmp_path[-1] 
        if last_node == end: 
            return tmp_path 
        for next_node in graph.children_of(last_node): 
            if next_node not in tmp_path: 
                new_path = tmp_path + [next_node] 
                path_queue.append(new_path) 
    return None 

In [15]:
def test_SP(): 
    nodes = [] 
    for name in range(6): #6개 노드를 만듭니다
        nodes.append(Node(str(name))) 
    g = Digraph() 
    for n in nodes: 
        g.add_node(n) 
    g.add_edge(Edge(nodes[0],nodes[1])) 
    g.add_edge(Edge(nodes[1],nodes[2])) 
    g.add_edge(Edge(nodes[2],nodes[3])) 
    g.add_edge(Edge(nodes[2],nodes[4])) 
    g.add_edge(Edge(nodes[3],nodes[4])) 
    g.add_edge(Edge(nodes[3],nodes[5])) 
    g.add_edge(Edge(nodes[0],nodes[2])) 
    g.add_edge(Edge(nodes[1],nodes[0])) 
    g.add_edge(Edge(nodes[3],nodes[1])) 
    g.add_edge(Edge(nodes[4],nodes[0])) 
    sp = BFS(g, nodes[0], nodes[5], to_print=True) 
    print('BFS가 찾은 최단 경로:', print_path(sp)) 

test_SP()

현재 BFS 경로: 0
현재 BFS 경로: 0->1
현재 BFS 경로: 0->2
현재 BFS 경로: 0->1->2
현재 BFS 경로: 0->2->3
현재 BFS 경로: 0->2->4
현재 BFS 경로: 0->1->2->3
현재 BFS 경로: 0->1->2->4
현재 BFS 경로: 0->2->3->4
현재 BFS 경로: 0->2->3->5
BFS가 찾은 최단 경로: 0->2->3->5
